In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import datetime

from uszipcode import SearchEngine
from uszipcode import Zipcode

import random
import scipy.stats as stats

import holidays

%matplotlib inline
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.dates as mdates

colors = mcolors.TABLEAU_COLORS

import warnings
warnings.filterwarnings("ignore")

# Data import

#### Import station data

In [ ]:
station_data = pd.read_csv('data/station_data.csv')
print(station_data.shape)
print(station_data.tail())

In [ ]:
print(station_data.dtypes)

In [ ]:
station_data.isnull().sum()

#### import trip data

In [ ]:
trip_data = pd.read_csv('data/trip_data.csv')
print(trip_data.shape)
print(trip_data.head())
print(trip_data.dtypes)

In [ ]:
trip_data.isnull().sum()

#### Import weather data

In [ ]:
weather_data = pd.read_csv('data/weather_data.csv')
print(weather_data.shape)
print(weather_data.head())

In [ ]:
print(weather_data.dtypes)

In [ ]:
weather_data.isnull().sum()

# Exploratory data analysis

First we link all three data sets to leverage all information contained in th dataset. We start by including zip codes in the station_data to be able to link to the weather data.

Add city names to the trips data

In [ ]:
def get_city_name(x):
    return station_data[station_data.Id == x].City.values[0]

trip_data["Pick_up_city"] = trip_data["Start Station"].apply(lambda x: get_city_name(x))
trip_data["Drop_off_city"] = trip_data["End Station"].apply(lambda x: get_city_name(x))

We create time dependent variables and flag business days and holidays

In [ ]:
trip_data["Start_Date_time"] =  pd.to_datetime(trip_data["Start Date"], dayfirst=True)
trip_data["End_Date_time"] =  pd.to_datetime(trip_data["End Date"], dayfirst=True)

trip_data["Start_Date"] =  trip_data["Start_Date_time"].dt.date
trip_data["End_Date"] =  trip_data["End_Date_time"].dt.date

trip_data['Start_month'] = trip_data['Start_Date_time'].dt.month
trip_data['Start_weekday'] = trip_data['Start_Date_time'].dt.dayofweek
trip_data['Start_hour'] = trip_data['Start_Date_time'].dt.hour

trip_data['End_month'] = trip_data['End_Date_time'].dt.month
trip_data['End_weekday'] = trip_data['End_Date_time'].dt.dayofweek
trip_data['End_hour'] = trip_data['End_Date_time'].dt.hour

trip_data['is_weekend_start'] = trip_data.Start_weekday.isin([5,6])*1
trip_data['is_weekend_end'] = trip_data.End_weekday.isin([5,6])*1

# Reset date time to top of the hour
trip_data["Start_Date_hour"] = trip_data['Start_Date_time'].values.astype('<M8[h]')
trip_data["End_Date_hour"] = trip_data['End_Date_time'].values.astype('<M8[h]')


In [ ]:
import holidays
holidays =  holidays.US(state='CA', years=[2014,2015])

weekDays = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}

def set_holidays_to_7(x, holidays=holidays):
    if x in holidays:
        return 7
    else:
        return x.weekday()
            
def set_names(x, holidays=holidays):
    return weekDays[x.weekday()]

def check_holidays(x, holidays=holidays):
    if x in holidays:
        return True
    else:
        return False

def check_business_day(x, holidays=holidays):
    if x in holidays or x.weekday() in [5,6]:
        return False
    else:
        return True

trip_data["holiday"] = trip_data["Start_Date"].apply(lambda x: check_holidays(x))
trip_data["Business_day"] = trip_data["Start_Date"].apply(lambda x: check_business_day(x))
trip_data["Start_weekday_no"] = trip_data["Start_Date"].apply(lambda x: set_holidays_to_7(x))
trip_data["Start_day_name"] = trip_data["Start_Date"].apply(lambda x: set_names(x))

### Explore trip per city and stations

In [ ]:
classes = station_data['City'].values
unique, counts = np.unique(classes, return_counts=True)
plt.figure(figsize=(8,6))
plt.bar(unique,counts, color=colors)
plt.title('Number of stations per city')
plt.xlabel('City')
plt.ylabel('Number of stations')
plt.show()

In [ ]:
# View the number of trips on each month.
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
classes = trip_data['Subscriber Type'].values
unique, counts = np.unique(classes, return_counts=True)
axes[0].bar(unique,counts, color=colors)
axes[0].set_xlabel('Subscriber Type')
axes[0].set_ylabel('Number of trips')
axes[0].set_title('Subscriber Type Frequency')

classes = station_data['City'].values
unique, counts = np.unique(classes, return_counts=True)
axes[1].bar(unique,counts, color=colors)
axes[1].set_xlabel('City')
axes[1].set_ylabel('Number of trips')
axes[1].set_title('Number of stations per city')
plt.show()

In [ ]:
trip_data_out = trip_data.loc[trip_data['Pick_up_city']!= trip_data['Drop_off_city']]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
classes = trip_data_out['Pick_up_city'].values
unique, counts = np.unique(classes, return_counts=True)
axes[0].bar(unique,counts, color=colors)
axes[0].set_xlabel('City')
axes[0].set_ylabel('Number of trips')
axes[0].set_title('Out of city Start Station')

classes = trip_data_out['Drop_off_city'].values
unique, counts = np.unique(classes, return_counts=True)
axes[1].bar(unique,counts, color=colors)
axes[1].set_xlabel('City')
axes[1].set_ylabel('Number of trips')
axes[1].set_title('Out of city End Station')
plt.show()


In [ ]:
for city in station_data.City.unique():
    total_trips = trip_data[trip_data.Pick_up_city == city].shape[0]
    num_out_of_city = trip_data_out[trip_data_out.Pick_up_city == city].shape[0]
    ratio = num_out_of_city/total_trips
    print('The percentage of trips from a start station in {} to a different city is {:.3f}% out of total {} trips.'.format
          (city, ratio, total_trips))

In [ ]:
classes = trip_data_out['Subscriber Type'].values
unique, counts = np.unique(classes, return_counts=True)
plt.figure(figsize=(8,6))
plt.bar(unique,counts, color=colors)
plt.title('Subscriber Type Frequency')
plt.xlabel('Subscriber Type')
plt.ylabel('Number of trips')
plt.show()

In [ ]:
classes = trip_data['Start Station'].values
unique_start, counts_start = np.unique(classes, return_counts=True)
classes = trip_data['End Station'].values
unique_end, counts_end = np.unique(classes, return_counts=True)

plt.figure(figsize=(24,10))
plt.title('Pict-up/drop-off frequency per station', fontsize=18)
plt.scatter(unique_start,counts_start,label='Pick up')
plt.scatter(unique_end,counts_end, label='Drop off')

plt.xlabel('Station ID', fontsize=20)
plt.ylabel('Counts', fontsize=20)

plt.tick_params(labelsize=11)
plt.xticks(unique_start)
plt.legend(fontsize=18)
plt.show()

In [ ]:
for i, row in station_data.iterrows():
    start_trips = len(trip_data[trip_data['Start Station'] == row.Id])
    end_trips = len(trip_data[trip_data['End Station'] == row.Id])
    station_data.at[i, "net_rate"] = end_trips - start_trips
    
fig = plt.figure(figsize=(20,10))
fig.suptitle('Net rate per City', fontsize=20)

i = 231
for city in set(station_data.City):
    tmp_df = station_data[station_data.City == city]
    pos_signal = tmp_df.net_rate.copy()
    neg_signal = tmp_df.net_rate.copy()
    pos_signal[pos_signal <= 0] = np.nan
    neg_signal[neg_signal > 0] = np.nan
    
    x_pos = [i for i, _ in enumerate(tmp_df.Id)]
    
    ax = fig.add_subplot(i)
    ax.set_title(city)
    ax.bar(x_pos, pos_signal, color='b', align='center')
    ax.bar(x_pos, neg_signal, color='r', align='center')
    plt.xticks(x_pos, tmp_df.Id)
    
    i = i + 1

With above net rate distribution shows that the pick up and drop off varies across stations. Some stations seem to get more bike dropped off than they are picked up indicating that the location of the station might be important in predicting the net rental rates.

### Explore trips by days and hours

In [ ]:
print('There are a total of {} business days in the time period under investigation.'.format
      (len(trip_data[trip_data['Business_day']==True].Start_Date.unique())))
print('There are a total of {} days in the time period under investigation.'.format(len(trip_data['Start_Date'].unique())))

In [ ]:
# number of trips per weekday and holidays
weekDays = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday", 'Holiday']
val = [0,1,2,3,4,5,6,7]
plt.subplots(figsize=(10, 6))
trip_data.groupby('Start_month')['Trip ID'].count().plot('bar')
plt.xlabel("Months")
plt.ylabel("Number of trips")
plt.xticks(rotation='horizontal')
plt.show()

In [ ]:
weekDays = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday", 'Holiday']
val = [0,1,2,3,4,5,6,7]
plt.subplots(figsize=(10, 6))
trip_data.groupby('Start_weekday_no')['Trip ID'].count().plot('bar')
plt.xlabel("Days")
plt.ylabel("Number of trips")
plt.xticks(val, weekDays, rotation='horizontal')
plt.show()

In [ ]:
plt.subplots(figsize=(10, 6))
trip_data.groupby('Start_hour')['Trip ID'].count().plot('bar')
plt.title("Start hour Distribution ")
plt.xticks(rotation='horizontal', fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel("Hours", fontsize=18)
plt.ylabel("Frequency", fontsize=18)
plt.show()

In [ ]:
plt.subplots(figsize=(10, 6))
trip_data.groupby('End_hour')['Trip ID'].count().plot('bar')
plt.title("End hour Distribution ")
plt.xticks(rotation='horizontal', fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel("Hours", fontsize=18)
plt.ylabel("Frequency", fontsize=18)
plt.show()

In [ ]:
def plot_freq_by_subscription(var):
    
    weekDays = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    
    fig = plt.figure(figsize=(20,20))
    fig.subplots_adjust(wspace=0.3, hspace=0.6)
    fig.tight_layout()
    title = 'Trip frequency by ' + var + ' per City'
    fig.suptitle(title, fontsize=20)

    i = 1
    for city in set(station_data.City):
        tmp_trip = trip_data[trip_data.Pick_up_city == city]
        ax = fig.add_subplot(5, 3, i)
        tmp_trip.groupby(var)['Trip ID'].count().plot('bar')
        ax.set_title(city)
        i = i+1
        for subs in set(trip_data['Subscriber Type']):
            tmp_trip = trip_data.loc[(trip_data['Pick_up_city'] == city) & (trip_data['Subscriber Type'] == subs)]
            ax = fig.add_subplot(5, 3, i)
            if 'Start_day' in var:
                tmp_trip.set_index(var).loc[weekDays].groupby(var)['Trip ID'].plot('bar')
            else:
                tmp_trip.groupby(var)['Trip ID'].count().plot('bar')
            
            title2 = city + " (" + subs + ")"
            ax.set_title(title2)
            i = i + 1 

In [ ]:
plot_freq_by_subscription('Start_hour') 

In [ ]:
# Day 7 is attributed to all holidays
plot_freq_by_subscription('Start_weekday_no')

In [ ]:
plot_freq_by_subscription('Start_month')

Explore the rental frequency over time across all stations. 

In [ ]:
daily_trips = trip_data.groupby('Start_Date').agg({'Business_day': ['median', 'count']})

BDay_trips = daily_trips[daily_trips['Business_day']['median'] == True].reset_index()
nonBDay_trips = daily_trips[daily_trips['Business_day']['median'] == False].reset_index()

fig, ax = plt.subplots(figsize=(20, 8))
plt.plot(BDay_trips.Start_Date, BDay_trips['Business_day']['count'], 'bo', label='Business Day')
plt.plot(nonBDay_trips.Start_Date, nonBDay_trips['Business_day']['count'], 'ro', label='Non-business Day')

months = mdates.MonthLocator()
year_month_Fmt = mdates.DateFormatter('%y/%m')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(year_month_Fmt)

plt.xticks(rotation='horizontal', fontsize=16)
plt.yticks(fontsize=18)
plt.xlabel("Days", fontsize=18)
plt.legend(fontsize=18)
plt.ylabel('Trip Counts', fontsize=18)

plt.show()

### Rental duration

In [ ]:
trip_data['Trip_duration'] = trip_data["End_Date_time"] - trip_data["Start_Date_time"]
trip_data['Trip_duration']=trip_data['Trip_duration']/np.timedelta64(1,'h')
#trip_data['Trip_duration']=trip_data['Trip_duration']/np.timedelta64(1,'m')

print(trip_data['Trip_duration'].describe())
print()
print('Average ride time (hours) {} .'.format(trip_data['Trip_duration'].mean()))

In [ ]:
plt.figure(figsize=(16, 6))
sns.distplot(trip_data['Trip_duration'], hist=False, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})

We observe that most of the trips are under 24hrs with an average duration of about 20mins. we can define a rudimentary threshold to eliminate possible outling trips using the simple mead + 2*std as the cutoff and all trips about 

In [ ]:

window = trip_data['Trip_duration'].mean() + 2*trip_data['Trip_duration'].std()
print('Threshold of outlying trips (hours) {} .'.format(window))

trip_data_out = trip_data[trip_data['Trip_duration'] <= window]

plt.figure(figsize=(16, 6))
x = trip_data_out.Trip_duration
sns.distplot(x, hist=False, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})

In [ ]:
trip_duration_subscriber = trip_data[trip_data['Subscriber Type']=='Subscriber'].Trip_duration.mean()
trip_duration_customer = trip_data[trip_data['Subscriber Type']=='Customer'].Trip_duration.mean()

print('The average trip duration for a subscriber is %.1f hours.' % trip_duration_subscriber)
print('The average trip duration for a non-subscriber is %.1f hours.' % trip_duration_customer)

In [ ]:
for city in station_data.City.unique():
    duration = trip_data[trip_data['Pick_up_city']==city].Trip_duration.mean()
    print('The average trip duration for {} is {:.2f}. hours'.format(city, duration))

### Evaluate effect of station change

In [ ]:
trips_changed_station = {}
stations = [23, 25, 49, 69, 72, 85, 86, 87, 88, 89, 90]
hours = set(trip_data.Start_hour)

station_list = []
hour_list = []
trip_start_list = []
trip_end_list = []
net_rate_list = []

for station in stations:
    for hour in range(24):
        trip_start = len(trip_data[(trip_data['Start Station'] == station) & (trip_data['Start_hour'] == hour)])
        trip_end = len(trip_data[(trip_data['End Station'] == station) & (trip_data['End_hour'] == hour)])
        net_rate = trip_end - trip_start
        
        trip_start_list.append(trip_start)
        trip_end_list.append(trip_end)
        station_list.append(station)
        hour_list.append(hour)
        net_rate_list.append(net_rate)
        
# intialise data of lists. 
data = {'Start_station':station_list, 'hour':hour_list, 'trip_starts': trip_start_list, 
        'trip_ends':trip_end_list,'net_rate': net_rate_list} 
df = pd.DataFrame(data) 

In [ ]:
import scipy.stats as stats
from scipy.stats import wilcoxon
station_change = {
    23: 85,
    25: 86,
    49: 87,
    69: 88,
    72: 89,
    89: 90,
    90: 72
}

for station1, station2 in station_change.items(): 
    print("Comparison between ", str(station1), " vs ", str(station2))
    group1 = df[df.Start_station == station1].net_rate.tolist()
    group2 = df[df.Start_station == station2].net_rate.tolist()
    u_statistic, pVal = wilcoxon(group1, group2)
    print ('P value: ' + str(pVal))

In [ ]:
station_change = {
    23: 85,
    25: 86,
    49: 87,
    69: 88,
    72: 89,
    89: 90
}

fig = plt.figure(figsize=(20,15))
fig.subplots_adjust(wspace=0.3, hspace=0.6)
fig.tight_layout()

i = 1
for station1, station2 in station_change.items():
    
    hour = df[df.Start_station == station1].hour.tolist()
    group1 = df[df.Start_station == station1].net_rate.tolist()
    group2 = df[df.Start_station == station2].net_rate.tolist()
    
    ax = fig.add_subplot(3, 2, i)
    title = 'Net rate for ' + str(station1) + ' vs ' + str(station2)
    plt.title(title)
    plt.plot(hour, group1,label=station1, color='r')
    plt.plot(hour, group2,label=station2, color='b')
    plt.xlabel('Hour')
    plt.ylabel('Net rate')
    plt.xticks(hour)
    plt.legend()
    i = i + 1 

In [ ]:
fig = plt.figure(figsize=(20,15))
fig.subplots_adjust(wspace=0.3, hspace=0.6)
fig.tight_layout()

i = 1
for station1, station2 in station_change.items():
    
    hour = df[df.Start_station == station1].hour.tolist()
    group1 = df[df.Start_station == station1].trip_ends.tolist()
    group2 = df[df.Start_station == station2].trip_ends.tolist()
    
    ax = fig.add_subplot(3, 2, i)
    title = 'Trip end count for ' + str(station1) + ' vs ' + str(station2)
    plt.title(title)
    plt.plot(hour, group1,label=station1, color='r')
    plt.plot(hour, group2,label=station2, color='b')
    plt.xlabel('Hour')
    plt.ylabel('Net rate')
    plt.xticks(hour)
    plt.legend()

    i = i + 1 

There in no strong statistically significant evidence that the move station have impact on the net rate distribution. Bas on this we update the station in the trip data in order to have a complete time series flow for all stations

In [ ]:
### update station info for moved stations
station_update = {
    23: 85,
    25: 86,
    49: 87,
    69: 88,
    89: 90,
    72: 90
}

def value_to_update(x):
    if x in station_update:
        return station_update[x]
    else:
        return x
    
trip_data["Start Station"] = trip_data["Start Station"].apply(lambda x: value_to_update(x))
trip_data["End Station"] = trip_data["End Station"].apply(lambda x: value_to_update(x))

#### Explore weather data

In [ ]:
weather_data["Date_time"] =  pd.to_datetime(weather_data["Date"], dayfirst=True)
weather_data['Date_str'] = weather_data.Date_time.astype(str)
weather_data['Month'] = weather_data['Date_time'].dt.month
weather_data['Weekday'] = weather_data['Date_time'].dt.dayofweek
weather_data["Date_time"] =  pd.to_datetime(weather_data["Date"], dayfirst=True).dt.date

In [ ]:
for col in weather_data.columns:
    if pd.api.types.is_numeric_dtype(weather_data[col]):
        weather_data[col].fillna(weather_data.groupby("Zip")[col].transform("mean"), inplace=True)
        

In [ ]:
#We assume if there is an nan in the event then it is most likely a day without rain fog or Thunderstorm
print(weather_data.Events.unique())
weather_data['Events'].fillna('Clear', inplace=True)

In [ ]:
# We assume non gust values imply there was no wind gust and thus we set it to zero
weather_data.loc[weather_data['Max Gust SpeedMPH'] == 0, 'Max Gust SpeedMPH']

In [ ]:
weather_data['Max Gust SpeedMPH'].fillna(0, inplace=True)

In [ ]:
# Create a correlation table between all the features.
weather_corr_table = weather_data.corr(method='pearson', min_periods=1)
display(weather_corr_table)

In [ ]:
weather_data.columns

In [ ]:
corr_cols = ['Max TemperatureF', 'Mean TemperatureF', 'Min TemperatureF','Max Dew PointF', 'MeanDew PointF', 'Min DewpointF', 
             'Max Humidity','Mean Humidity', 'Min Humidity', 'Max Sea Level PressureIn','Mean Sea Level PressureIn', 
             'Min Sea Level PressureIn','Max VisibilityMiles', 'Mean VisibilityMiles', 'Min VisibilityMiles',
             'Max Wind SpeedMPH', 'Mean Wind SpeedMPH', 'Max Gust SpeedMPH','PrecipitationIn', 'CloudCover', 'WindDirDegrees']

plt.figure(figsize=(10,10))
corr = weather_data[corr_cols].corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);


In [ ]:
corr_matrix = weather_data.corr(method='pearson', min_periods=1)
r_squared = 0.5

corr = []
for col in corr_matrix.columns:
    feature = corr_matrix[col]
    corr_cols = feature[(feature.pow(2) > r_squared) & (feature != 1)]
    corr_idx = corr_cols.index.values.tolist()
    corr_dict = {col: corr_idx}
    if corr_idx != []:
        corr.append(corr_dict)

In [ ]:
display(corr)

In [ ]:
weather_data.columns

In [ ]:
# remove highly correlated colunns
selected_cols = ['Date', 'Mean TemperatureF', 'Mean Humidity', 'Mean Sea Level PressureIn','Max VisibilityMiles', 
                 'Min VisibilityMiles','Max Wind SpeedMPH', 'Max Gust SpeedMPH','PrecipitationIn', 'CloudCover', 
                 'Events', 'WindDirDegrees', 'Zip','Date_time', 'Date_str', 'Month', 'Weekday']
#weather_data = weather_data[selected_cols]

As expected there is a very high correlation between the min, mean and max values for each weather measurement.  
Also there is a significant correlation between temperature and Due point measures  


In [ ]:
daily_temp = pd.DataFrame(weather_data.groupby('Date')['Mean TemperatureF'].mean())
daily_temp.index = pd.to_datetime(daily_temp.index, dayfirst=True)

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(daily_temp.index, daily_temp['Mean TemperatureF'], 'bo')
months = mdates.MonthLocator()
year_month_Fmt = mdates.DateFormatter('%y/%m')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(year_month_Fmt)
plt.xticks(rotation='horizontal')
plt.xlabel("Dates")
plt.ylabel('Temperature')
plt.show()

### Aggregate the trips over each out to calculate net rate per hour

In [ ]:
def bin_time_num(x):
    if x.hour in [6,7,8,9]:
        return datetime.datetime(x.year, x.month, x.day, 1, 0, 0)
    elif x.hour in [10,11,12,13,14]:
        return datetime.datetime(x.year, x.month, x.day, 2, 0, 0)
    elif x.hour in [15,16,17,18,19]:
        return datetime.datetime(x.year, x.month, x.day, 3, 0, 0)
    else:
        return datetime.datetime(x.year, x.month, x.day, 4, 0, 0)

def bin_time(x):
    if x.hour in [6,7,8,9,15,16,17,18,19]:
        return "peak"
    elif x.hour in [10,11,12,13,14]:
        return "low peak"
    else:
        return "off peak"

trip_data["Start_Date_time_bin"] = trip_data["Start_Date_time"].apply(lambda x: bin_time_num(x))
trip_data["End_Date_time_bin"] = trip_data["End_Date_time"].apply(lambda x: bin_time_num(x))

In [ ]:
# drop outlining trips
trip_data = trip_data[trip_data['Trip_duration'] <= window]

In [ ]:
  
def binary_day(x, holidays=holidays):
    x = pd.to_datetime(x)
    holiday = 0
    bday = 1
    weekday = 1
    
    if x in holidays:
        holiday = 1
    
    if x in holidays or x.weekday in [5,6]:
        bday = 0
        
    if x.weekday in [5,6]:
        weekday = 0
    
    return bday, holiday, weekday

    
def month_day_hour(x):
    #x = x.to_datetime(x)
    return x.month, x.weekday(), x.hour


def aggregate_station_level(trip_df, station_df, weather_df, start_time, end_time):
    
    zipcode = {'Redwood City' : 94063,'San Francisco': 94107,'Palo Alto': 94301,'Mountain View': 94041,'San Jose': 95113}

    date_list = []
    station_list = []
    city_list = []
    hour_list = []
    month_list = []
    day_list = []
    net_rate_list = []
    holiday_list = []
    Business_day_list = []
    weekday_list = []
    lat_list = []
    long_list = []
    zip_list = []
    weather_list = []
    
    stations = station_df.Id.unique()
    cities = station_df.City.unique()

    for city in cities:
        for station in stations:
            
            tmp_trip_start = trip_df[(trip_df.Pick_up_city == city) & (trip_df['Start Station'] == station)]
            tmp_trip_end = trip_df[(trip_df.Pick_up_city == city) & (trip_df['End Station'] == station)]
            
            dates = tmp_trip_start[start_time].append(tmp_trip_end[end_time])
            dates = pd.to_datetime(dates.sort_values(ascending=True).unique())           
            
            station_zip = zipcode[station_data[station_data.Id == station].City.values[0]]
        
            for date in dates:
                trip_start = tmp_trip_start[tmp_trip_start[start_time] == date]
                trip_end = tmp_trip_end[tmp_trip_end[end_time] == date]
                
                date_to_str = str(date.date())
                weather_of_day = weather_df[(weather_df['Date_str'].str.contains(date_to_str)) & 
                                              (weather_df.Zip == int(station_zip))]
                try:
                    weather_list.append(weather_of_day.values.tolist()[0])
                except:
                    weather_list.append([])
                
                zip_list.append(station_zip)
                city_list.append(city)
                station_list.append(station)
                
                lat_list.append(station_df[station_df.Id == station].Lat.tolist()[0])
                long_list.append(station_df[station_df.Id == station].Long.tolist()[0])
                date_list.append(pd.to_datetime(date))

                bday, holiday, weekday = binary_day(date)
                Business_day_list.append(bday)
                holiday_list.append(holiday)
                weekday_list.append(weekday)

                net_rate = trip_end.shape[0] - trip_start.shape[0]
                net_rate_list.append(net_rate)
                
                month, day, hour = month_day_hour(date)
                month_list.append(month)
                day_list.append(day)
                hour_list.append(hour)
    
    
    df = pd.DataFrame({'City':city_list,'Station':station_list,'Date':date_list,'Lat':lat_list, 'Long':long_list,
                       'Business_day':Business_day_list, 'holiday':holiday_list, 'is_weekday':weekday_list, 'hour':hour_list,
                       'month':month_list, 'day':day_list, 'net_rate':net_rate_list})
    
    df_w = pd.DataFrame(weather_list, columns = list(weather_df.columns))
    trips = pd.concat([df, df_w], axis = 1)
    
    return trips

In [ ]:
trip_data_hour = aggregate_station_level(trip_data, station_data, weather_data, 'Start_Date_hour','End_Date_hour')

In [ ]:
plt.scatter(trip_data_hour.net_rate, trip_data_hour['Mean TemperatureF'])
plt.xlabel('Net rate')
plt.ylabel('Mean Temperature')
plt.title('Scatter plot Net rate vs Mean Temperature')

In [ ]:
plt.scatter(trip_data_hour.net_rate, trip_data_hour['Mean Sea Level PressureIn'])
plt.xlabel('Net rate')
plt.ylabel('Mean Sea Level PressureIn')
plt.title('Scatter plot Net rate vs Mean Sea Level PressureIn')

In [ ]:
fig = plt.figure(figsize=(20,15))
fig.subplots_adjust(wspace=0.3, hspace=0.6)
fig.tight_layout()

i = 1
for city in station_data.City.unique():
    
    ax = fig.add_subplot(3, 2, i)
    n, bins, patches = plt.hist(trip_data_hour[trip_data_hour.City == city].net_rate, 100, 
                                density=True, facecolor='g', alpha=0.75)
    plt.xlabel('Net rate')
    plt.ylabel('Probability')
    plt.title(city)

    i = i + 1

The net rate distribution for the are similar in all towns except San Fracisco indicating that we might group these cities and develop a single prediction model for them and another for San Francisco.

In [ ]:
from scipy.stats import kruskal

cities = ['San Francisco', 'San Jose', 'Redwood City', 'Mountain View', 'Palo Alto']

Francisco = trip_data_hour[trip_data_hour.City == 'San Francisco'].net_rate.tolist()
Jose = trip_data_hour[trip_data_hour.City == 'San Jose'].net_rate.tolist()
Redwood = trip_data_hour[trip_data_hour.City == 'Redwood City'].net_rate.tolist()
Mountain = trip_data_hour[trip_data_hour.City == 'Mountain View'].net_rate.tolist()
Palo = trip_data_hour[trip_data_hour.City == 'Palo Alto'].net_rate.tolist()
u_statistic, pVal = kruskal(Francisco, Jose, Redwood, Mountain, Palo)
print("P value of combined test " + str(pVal) + "\n")

n = len(cities)
for i in range(len(cities)):
    city_i = trip_data_hour[trip_data_hour.City == cities[i]].net_rate.tolist()
    for j in range(i+1,n):
        city_j = trip_data_hour[trip_data_hour.City == cities[j]].net_rate.tolist()
        print("Comparison between " + cities[i] + " vs " + cities[j])
        u_statistic, pVal = kruskal(city_i, city_j)
        print ('P value: ' + str(pVal))

# Regression model

First we create a moving average as a baseline model to evaluate our model performance

In [ ]:
trip_data_hour = pd.read_csv('data/trip_data_hour_with_weather.csv')
trip_data_hour.head()

In [ ]:
def root_mean_squared_error(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])

cities = trip_data_hour.City.unique()
for city in cities:
    ts = trip_data_hour[trip_data_hour.City == city]
    ts = ts.set_index('Date')
    avg = moving_average(ts.net_rate, 1)
    print("Moving average for " + city + " is " + str(avg))

In [ ]:
def plotMovingAverage(series, window, city, plot_intervals=False, scale=2, plot_anomalies=False):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        rmse = root_mean_squared_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (rmse + scale * deviation)
        upper_bond = rolling_mean + (rmse + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond")
        plt.plot(lower_bond, "r--", label="Lower Bond")
        
        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            plt.plot(anomalies, "ro", markersize=10)
        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [ ]:
cities = trip_data_hour.City.unique()
cities

### Moving average for San Francisco

In [ ]:
#window = 4
#cities = trip_data_hour.City.unique()
city = 'San Jose'
ts = trip_data_hour[trip_data_hour.City == city]
stnt = ts.Station.unique()[0]
ts = ts[ts.Station == stnt]
ts = ts.set_index('Date')
    
for window in [2,4,8]:
    plotMovingAverage(ts['2014-12-01':'2014-12-14'].net_rate, window, city, plot_intervals=True)

we use latitude and longitude to account for the different stations and drop the station ID.  
We develop 2 models one for san Francisco and another for the other stations  

In [ ]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.svm import LinearSVR, SVR
from xgboost import XGBRegressor

from numpy import sort
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel

In [ ]:
# Encoding cyclic feature
trip_data_hour['hour'] = np.sin(trip_data_hour.hour*(2.*np.pi/24))
trip_data_hour['day'] = np.sin(trip_data_hour.day*(2.*np.pi/7))
trip_data_hour['month'] = np.sin((trip_data_hour.month-1)*(2.*np.pi/12))

In [ ]:
trip_data_hour = pd.get_dummies(trip_data_hour, columns=["Events"])

In [ ]:
def group_city(x):
    if x != 'San Francisco':
        return "Others"
    else:
        return x

trip_data_hour["Region"] = trip_data_hour["City"].apply(lambda x: group_city(x))

In [ ]:
def root_mean_squared_error(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

random_state = 12

In [ ]:
cols_to_drop = ['City', 'Station', 'Date','Zip', 'Date_time', 'Date_str', 'Month', 'Weekday', 'Region']

numeric_cols =  ['Mean TemperatureF', 'Mean Humidity', 'Mean Sea Level PressureIn','Max Gust SpeedMPH','PrecipitationIn', 
                 'CloudCover', 'WindDirDegrees']

#numeric_cols =  ['Mean TemperatureF', 'Mean Humidity', 'Mean Sea Level PressureIn','Max VisibilityMiles','Min VisibilityMiles',
#                 'Max Wind SpeedMPH', 'Max Gust SpeedMPH','PrecipitationIn', 'CloudCover', 'WindDirDegrees']

# 'Min VisibilityMiles', 'Max Wind SpeedMPH', 'Max VisibilityMiles'

### San Francisco Model 

In [ ]:
Francisco = trip_data_hour[trip_data_hour.Region == 'San Francisco']

#### Set aside the data for the last month (August 2015) as the test set

In [ ]:
Francisco_test = Francisco[Francisco.Date_time >= '2015-08-01']
Francisco = Francisco[Francisco.Date_time < '2015-08-01']

In [ ]:
Francisco = trip_data_hour[trip_data_hour.Region == 'San Francisco']
Francisco = Francisco.drop(cols_to_drop, axis=1)
    
X = Francisco.drop(['net_rate'], axis=1)
y = Francisco.net_rate

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=random_state)

scaler = StandardScaler()
scaler.fit(Xtrain[numeric_cols])

Xtrain[numeric_cols] = scaler.transform(Xtrain[numeric_cols])
Xtest[numeric_cols] = scaler.transform(Xtest[numeric_cols])

In [ ]:
regressors = [GradientBoostingRegressor, RandomForestRegressor,Lasso, LinearRegression, Ridge, LinearSVR, SVR, XGBRegressor]
X_train, X_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=random_state)
    
rgs_dict = {}
for rgs in regressors:
    try:
        rgs_function = rgs(random_state=random_state) 
    except:
        rgs_function = rgs()
        
    model = rgs_function.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(root_mean_squared_error(y_pred, y_test))
    rgs_dict[rgs.__name__] = rmse

best_reg_model = min(rgs_dict, key = lambda x: rgs_dict.get(x))
print('The best regressor is {} with root mean squared error of {:.1f}.'.format(best_reg_model, rmse))

In [ ]:
pd.Series(rgs_dict).sort_values(ascending=False).plot.barh(figsize=(8, 6), grid=True, fontsize=12)
plt.xlabel('Root mean squared errors')
plt.show()

In [ ]:
rgs_dict

In [ ]:
model = GradientBoostingRegressor(random_state=random_state)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)

print("RMSE: %.2f" % rmse)

In [ ]:
feature_importance = pd.DataFrame({'Varaible':X_train.columns,
                                   'importance':model.feature_importances_}).sort_values(by='importance', ascending=False)
plt.subplots(figsize=(6, 8))
sns.barplot(x="importance", y="Varaible", data=feature_importance)
plt.title('Features Importance')

In [ ]:
thresholds = sort(model.feature_importances_)
thresholds = thresholds[::-1]
for thresh in thresholds:
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    selection_model = GradientBoostingRegressor(random_state=random_state)
    selection_model.fit(select_X_train, y_train)
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    print("Thresh=%.3f, n=%d, RMSE: %.2f" % (thresh, select_X_train.shape[1], rmse))

In [ ]:
best_feat = feature_importance.Varaible[:4].values

In [ ]:
gbr_params = {'learning_rate':[0.02, 0.05, 0.08], 'n_estimators':[150, 200, 250], 'min_samples_leaf':[3, 4, 5], 
              'max_depth':[8, 9, 10]}

nfolds = ShuffleSplit(n_splits=10, test_size = 0.20, random_state=random_state)

acc_scorer = make_scorer(root_mean_squared_error)
gbr = GradientBoostingRegressor(random_state=random_state) 
grid = GridSearchCV(estimator = gbr, param_grid = gbr_params, scoring = acc_scorer, cv = nfolds, n_jobs=2, verbose=1)
grid.fit(Xtrain[best_feat], ytrain.values)
grid.best_estimator_

In [ ]:
SF_model =  GradientBoostingRegressor(random_state=random_state)
SF_model.fit(X[best_feat], y)

In [ ]:
Francisco_test['net_rate_pred'] = SF_model.predict(Francisco_test[best_feat])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(ts['2015-08-01':'2015-08-31'].net_rate_pred, "g", label="predicted")
plt.plot(ts['2015-08-01':'2015-08-31'].net_rate, "b--", label="observed")

### Other regions Model 

In [ ]:
cols_to_drop = ['City', 'Station', 'Date','Zip', 'Date_time', 'Date_str', 'Month', 'Weekday', 'Region']

Others = trip_data_hour[trip_data_hour.Region == 'Others']
Others = Others.drop(cols_to_drop, axis=1)
    
X = Others.drop(['net_rate'], axis=1)
y = Others.net_rate
Xtrain_O, Xtest_O, ytrain_O, ytest_O = train_test_split(X, y, test_size=0.2, random_state=random_state)

scaler = StandardScaler()
scaler.fit(Xtrain_O[numeric_cols])

Xtrain_O[numeric_cols] = scaler.transform(Xtrain_O[numeric_cols])
Xtest_O[numeric_cols] = scaler.transform(Xtest_O[numeric_cols])

In [ ]:
Xtrain_O.head()

In [ ]:
regressors = [GradientBoostingRegressor, RandomForestRegressor,Lasso, LinearRegression, Ridge, LinearSVR, SVR, XGBRegressor]

X_train_O, X_test_O, y_train_O, y_test_O = train_test_split(Xtrain_O, ytrain_O, test_size=0.2, random_state=random_state)
    
rgs_dict_O = {}
for rgs in regressors:
    try:
        rgs_function = rgs(random_state=random_state) 
    except:
        rgs_function = rgs()
        
    model = rgs_function.fit(X_train_O, y_train_O.values)
    y_pred_O = model.predict(X_test_O)
    rmse_O = np.sqrt(root_mean_squared_error(y_pred_O, y_test_O))
    rgs_dict_O[rgs.__name__] = rmse_O
    
best_reg_model_O = min(rgs_dict_O, key = lambda x: rgs_dict_O.get(x))
print('The best regressor is {} with root mean squared error of {:.1f}.'.format(best_reg_model_O, rmse_O))

In [ ]:
rgs_dict_O

In [ ]:
pd.Series(rgs_dict_O).sort_values(ascending=False).plot.barh(figsize=(8, 6), grid=True, fontsize=12)
plt.xlabel('Root mean squared errors')
plt.show()

In [ ]:
from numpy import sort
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel

model = GradientBoostingRegressor(random_state=random_state)
X_train_O, X_test_O, y_train_O, y_test_O
model.fit(X_train_O, y_train_O)
y_pred = model.predict(X_test_O)
rmse = root_mean_squared_error(y_test_O, y_pred)

print("RMSE: %.2f" % rmse)

In [ ]:
feature_importance = pd.DataFrame({'Varaible':X_train_O.columns,
                                   'importance':model.feature_importances_}).sort_values(by='importance', ascending=False)
plt.subplots(figsize=(6, 8))
sns.barplot(x="importance", y="Varaible", data=feature_importance.head(30))
plt.title('Features Importance')

In [ ]:
thresholds = sort(model.feature_importances_)
thresholds = thresholds[::-1]
for thresh in thresholds:
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train_O)
    selection_model = GradientBoostingRegressor(random_state=random_state)
    selection_model.fit(select_X_train, y_train_O)
    select_X_test = selection.transform(X_test_O)
    y_pred = selection_model.predict(select_X_test)
    rmse = root_mean_squared_error(y_test_O, y_pred)
    print("Thresh=%.3f, n=%d, RMSE: %.2f" % (thresh, select_X_train.shape[1], rmse))

In [ ]:
best_feat = feature_importance.feature[:4]
best_feat

In [ ]:
gbr_params = {'learning_rate':[0.02, 0.05, 0.08], 'n_estimators':[150, 200, 250], 'min_samples_leaf':[3, 4, 5], 
              'max_depth':[8, 9, 10]}

nfolds = ShuffleSplit(n_splits=10, test_size = 0.20, random_state=random_state)

acc_scorer = make_scorer(root_mean_squared_error)
gbr_others = GradientBoostingRegressor(random_state=random_state) 
grid = GridSearchCV(estimator = gbr_others, param_grid = gbr_params, scoring = acc_scorer, cv = nfolds, n_jobs=2, verbose=1)
grid.fit(X_train_O[best_feat], y_train_O.values)

In [ ]:
grid.best_estimator_